In [9]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
from config import *
from tqdm import tqdm
import train_reg
import config
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# prepare data

In [40]:
dsets = ['clath_aux+gak_a7d2_new'] # this is one of the large datasets
feat_names = ['X_same_length', 'mean_total_displacement',
              'mean_square_displacement', 'lifetime']
meta = ['cell_num', 'Y_sig_mean']
df_full = None
for dset in dsets:
    df = data.get_data(dset=dset)
    df = df[df.lifetime > 15] # only keep hard tracks
    df = df[df.cell_num.isin(config.DSETS[dset]['train'])] # exclude held-out test data
    
    # downsample tracks
    length = 40
    df['X_same_length'] = [features.downsample(df.iloc[i]['X'], length)
                           for i in range(len(df))] # downsampling

    # regression response
    df = train_reg.add_sig_mean(df)     
    
    # remove extraneous feats
    df = df[feat_names + meta]
    df = df.dropna() 
    
    # merge all dsets
#     if df_full is None:
#         df_full = deepcopy(df)
#     else:
#         df_full = df_full.merge(df)

../src/train_reg.py:83: RuntimeWarning: invalid value encountered in less
  sigs = np.array(r[f'{track}_pvals']) < 0.05


## train neural net

In [36]:
# decide on architecture
out_dir = f'{DIR_RESULTS}/dec10_deep'
# out_dir = f'results/regression/deep_learning/Dec4'

In [37]:
os.makedirs(out_dir, exist_ok=True)
outcome_def = 'Y_sig_mean'
num_epochs = 100
num_hidden = 20
for model_type in [ 'nn_lstm', 'fcnn', 'nn_cnn']: # = 'nn_cnn' # 'nn_lstm', 'fcnn', 'nn_cnn', 'nn_attention'
    train_reg.train_reg(df,
                        feat_names=feat_names,
                        model_type=model_type, 
                        outcome_def=outcome_def,
                        out_name=oj(out_dir, f'{dset}_{outcome_def}_{model_type}.pkl'),
                        fcnn_hidden_neurons=num_hidden,
                        fcnn_epochs=num_epochs)

100%|██████████| 100/100 [00:37<00:00,  2.75it/s]


# analyze results

In [ ]:
results = train_reg.load_results(out_dir)
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r = r.sort_values(by=['r2'], ascending=False)
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
r